In [ ]:
@patch("rss_collector.orchestration.get_proxy_settings")
@patch("rss_collector.proxy.fetch_feed_content_with_proxy")
@patch("rss_collector.feed_parser.parse_feed")
def test_process_feeds_simplified(
    mock_parse_feed, mock_fetch_feed, mock_get_proxy_settings, mock_feed_data, mock_proxy_settings
):
    """
    Test process_feeds with mocked components to avoid external dependencies.
    """
    print("\nStarting test_process_feeds_simplified")

    # Mock proxy settings
    print("Mocking proxy settings...")
    mock_get_proxy_settings.return_value = mock_proxy_settings

    # Mock fetch_feed_content_with_proxy to return dummy content
    print("Mocking fetch_feed_content_with_proxy...")
    mock_fetch_feed.return_value = "Mock RSS Feed Content"

    # Mock parse_feed to return mock feed data
    print("Mocking parse_feed...")
    mock_parse_feed.return_value = mock_feed_data

    # Feed URLs for testing
    feed_urls = ["http://mock_feed_1.com/rss", "http://mock_feed_2.com/rss"]

    # Call process_feeds
    from rss_collector.orchestration import process_feeds
    process_feeds(feed_urls)

    # Debugging mock calls
    print("\nDebugging Mock Calls:")
    print(f"Mock Fetch Feed Calls: {mock_fetch_feed.call_count}")
    print(f"Mock Fetch Feed Call Args: {mock_fetch_feed.call_args_list}")
    print(f"Mock Parse Feed Calls: {mock_parse_feed.call_count}")
    print(f"Mock Parse Feed Call Args: {mock_parse_feed.call_args_list}")

    # Assertions
    # Assert proxy settings were called once
    mock_get_proxy_settings.assert_called_once()

    # Assert fetch_feed was called for each feed URL
    assert mock_fetch_feed.call_count == len(feed_urls), (
        f"Expected {len(feed_urls)} calls to mock_fetch_feed, but got {mock_fetch_feed.call_count}."
    )

    # Assert parse_feed was called for each feed URL
    assert mock_parse_feed.call_count == len(feed_urls), (
        f"Expected {len(feed_urls)} calls to mock_parse_feed, but got {mock_parse_feed.call_count}."
    )

    # Additional debugging for metadata file creation
    print("\nChecking metadata file creation...")
    metadata_files = os.listdir("rss_metadata")
    print(f"Metadata Files Found: {metadata_files}")
    assert len(metadata_files) == 1, "Expected one metadata file to be created."

    metadata_file = os.path.join("rss_metadata", metadata_files[0])
    with open(metadata_file, "r") as f:
        metadata_content = json.load(f)

    print(f"Metadata Content: {metadata_content}")
    assert "timestamp" in metadata_content, "Metadata file missing 'timestamp'."
    assert "articles" in metadata_content, "Metadata file missing 'articles'."
    assert len(metadata_content["articles"]) == len(feed_urls), (
        f"Expected {len(feed_urls)} articles in metadata, but got {len(metadata_content['articles'])}."
    )

    print("\nTest test_process_feeds_simplified completed successfully.")
